In [1]:
import copy
import os
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from scipy import stats
import pickle

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector
from tensorflow.keras.layers import Dense, Flatten, Reshape, Dropout
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [2]:
%run utils.ipynb

In [ ]:
def training_dense(train_data,
                   val_data,
                   n_layers_encoder = 1,
                   n_layers_decoder = 1,
                   hidden_units = [64, 32, 32, 64],
                   activation_hidden = 'relu',
                   activation_dense = 'sigmoid',
                   dropout_rate = 0,
                   learning_rate = 0.001,
                   n_epochs = 500,
                   metric = 'mse',
                   batch_size = 32,
                   plot = True,
                   save = True):

    number_of_features = train_data.shape[1]
    opt = Adam(learning_rate = learning_rate)

    # Build model
    inputs = Input(shape = (number_of_features))
    flattened = Flatten()(inputs)
    # Encoder
    encoded = Dense(hidden_units[0], activation = activation_hidden)(flattened)
    for i in range(0, n_layers_encoder):
        encoded = Dense(hidden_units[i], activation = activation_hidden)(encoded)
        if dropout_rate != 0:
            encoded = Dropout(dropout_rate)(encoded)
    # Decoder
    decoded = Dense(hidden_units[n_layers_encoder+1], activation = activation_hidden)(encoded)
    if dropout_rate != 0:
        decoded = Dropout(dropout_rate)(decoded)
    for i in range(0, n_layers_decoder):
        decoded = Dense(hidden_units[n_layers_encoder+i+2], activation = activation_hidden)(decoded)
        if dropout_rate != 0:
            decoded = Dropout(dropout_rate)(decoded)
    decoded = Dense(number_of_features, activation = activation_dense)(decoded)

    loss_metric = tf.keras.losses.MeanSquaredError()
    if metric == 'mse':
        loss_metric = tf.keras.losses.MeanSquaredError()
    elif metric == 'mae':
        loss_metric = tf.keras.losses.MeanAbsoluteError()

    # Compile model
    autoencoder = Model(inputs, decoded)
    autoencoder.compile(optimizer = opt, loss = loss_metric)

    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 100, verbose = 1, mode = 'min', restore_best_weights = True)

    # Train model
    if val_data is not None:
        history = autoencoder.fit(
            train_data, train_data,
            epochs = n_epochs,
            batch_size = batch_size,
            shuffle = False,
            validation_data = (val_data, val_data),
            callbacks = [early_stopping]
        )
    else:
        history = autoencoder.fit(
            train_data, train_data,
            epochs = n_epochs,
            batch_size = batch_size,
            shuffle = False
        )
    
    model_path = 'dense/' + str(n_layers_encoder) + '_' + str(n_layers_decoder) + '_' + str(hidden_units[0]) + '_' + str(hidden_units[len(hidden_units)-1]) + '_' + str(activation_hidden) + '_' + str(activation_dense) + '_' + str(dropout_rate).replace('.', '') + '_' + str(learning_rate).replace('.', '') + '_' + str(n_epochs) + '_' + str(metric) + '_' + str(batch_size)

    # Save history
    if plot:
        plt.figure(figsize = (10, 5))
        plt.plot(history.history['loss'], label = 'Training Loss')
        if 'val_loss' in history.history:
            plt.plot(history.history['val_loss'], label = 'Validation Loss')
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(loc = 'upper right')
        plt.tight_layout()
        loss_save_path = 'losses/' + model_path
        plt.savefig(loss_save_path + '.png')
        plt.close()
        
    # Save model
    if save:
        model_save_path = 'models/' + model_path + '.pkl'
        with open(model_save_path, 'wb') as file:
            pickle.dump(autoencoder, file)

    return history, autoencoder